# **Nearest Neighbour and Conformal Predicition applied to the Iris dataset from sklearn and the Ionoshphere dataset**

# Contents
# ~~~~~~
### 1. Functions for the assignment
### 2.a Iris data 1NN
### 2.b Ionosphere data 1NN
### 3.a Iris data conformal prediction
### 3.b Ionosphere data conformal prediciton
### 4 Results
# ~~~~~~

# Functions for the assignment

### Below are two functions that I will use in both the algorithms. min_finder will find the minimum value in a list or an array and argmin_finder will find the index of the minimum value in a list or array.

In [1]:
import math
def min_finder(x):
    """Finds the minimunm value of the list or array that is inputed"""
    min = math.inf
    for i in range (len(x)):
        if x[i] < min:
            min = x[i]
    return min

In [2]:
def argmin_finder(x):
    """Finds the minimunm value of the list or array that is inputed"""
    min = math.inf
    argmin = 0
    for i in range (len(x)):
        if x[i] < min:
            min = x[i]
            argmin = i
    return argmin

### euc_dis will be used in both algorithms to find the Euclidean distances between two vectors

In [3]:
import numpy as np
def euc_dis(x,y):
    """Return the Euclidean distance of the inputted vectors"""
    euc_calc = np.empty(len(x))
    for i in range(len(x)):
        euc_calc[i] =(x[i] - y[i]) ** 2
    ans = np.sum(euc_calc) ** 0.5
    return(ans)

# Iris data 1NN

### I imported the iris dataest and split it using sklearns train_test_split. 

In [4]:
from sklearn.datasets import load_iris

In [5]:
iris = load_iris()

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=1504)

### I then calculated the distances from every test point to every training point and put them all in the 2D array called distances.

In [8]:
distances = np.empty(len(X_test)*len(X_train)).reshape(len(X_test),len(X_train))
for i in range(len(X_test)):
    for j in range(len(X_train)):
        distances[i,j] = euc_dis(X_test[i],X_train[j])

### Finally I've predicted the label for each test data point based on the label given to the training point that is closest to it according to Euclidean distance. Then I've calculated how many of these predictions differ from the test data's actual labels to get the error rate.

In [9]:
y_pred = np.empty(len(X_test))
for i in range (len(X_test)):
    y_pred[i] = y_train[argmin_finder(distances[i])]

In [10]:
error_count = sum(y_pred!=y_test)
error_rate = (error_count/len(X_test))
error_rate

0.05263157894736842

1NN iris dataset test eror rate = 0.0526 (3.s.f)

# Ionoshpere data 1NN

### I loaded in the Ionosphere dataset using the np.genfromtxt and split the loaded data into its feature data and target data. The proccess after this is much the same as for the Iris dataest

In [11]:
ionosphere_data = np.genfromtxt('ionosphere.txt', delimiter=',', usecols=np.arange(34))
ionosphere_target = np.genfromtxt('ionosphere.txt', delimiter=',', usecols=34)

In [12]:
Xio_train, Xio_test, yio_train, yio_test = train_test_split(ionosphere_data, ionosphere_target, random_state=1504)

In [13]:
distances_io = np.empty(len(Xio_test)*len(Xio_train)).reshape(len(Xio_test),len(Xio_train))
for i in range(len(Xio_test)):
    for j in range(len(Xio_train)):
        distances_io[i,j] = euc_dis(Xio_test[i],Xio_train[j])

In [14]:
yio_pred = np.empty(len(Xio_test))
for i in range (len(Xio_test)):
    yio_pred[i] = yio_train[argmin_finder(distances_io[i])]

In [15]:
error_count_io = sum(yio_pred!=yio_test)
error_rate_io = (error_count_io/len(Xio_test))
error_rate_io

0.11363636363636363

1NN ionosphere dataset test error rate = 0.114 (3.s.f)

# Iris data Conformal Prediction

### Firstly I created a 3D matrix that contains every possible extended training dataset

In [16]:
big_train = np.empty((len(X_train)+1)*4*len(X_test)).reshape(len(X_test),(len(X_train)+1),4)
for i in range (len(X_test)):
    big_train[i] = np.r_[X_train,X_test[i].reshape(1,4)]

### In the code below I have looped through every extended training dataset and for each version I have calculated the distance from every point to every other point, making sure to not include the distance to itself as this would be 0 and cause problems further down the line. All of this is saved in the 3D matrix called big_distances. 

In [17]:
big_distances = np.empty(len(big_train)*len(big_train[0])*len(big_train[0])).reshape(len(big_train),len(big_train[0]),(len(big_train[0])))
for i in range (len(big_train)):
    for j in range (len(big_train[0])):
        for k in range (len(big_train[0])):
            if j != k:
                big_distances[i,j,k] = euc_dis(big_train[i,j],big_train[i,k])
            else:
                big_distances[i,j,k] = math.inf

### Next I turned this 3D matrix into a 4D matrix where each individual scalar represents either the minimum distance to the same class or the minimum distance to a different class of one of the data points on the extended training datasets where each test point on every extended training dataset either has the label  Y = {0,1,2}.

### For example min_dist_same_dif_class[0,0,0,0] represents the closest point of the same class to the first point in the extended training dataset where the test point that has been added to it is given the label 0.

In [18]:
y_train0 = np.append(y_train,0)
y_train1 = np.append(y_train,1)
y_train2 = np.append(y_train,2)
min_dist_same_dif_class = np.empty(2*3*len(big_train)*(len(big_train[0]))).reshape(2,3,len(big_train),(len(big_train[0])))

for ds in range (2):
    for a in range (3):
        for b in range (len(big_train)):
            for c in range (len(big_train[0])):
                if ds == 0:
                    if a == 0:
                        min_dist_same_dif_class[ds,a,b,c] = min_finder(big_distances[b,c][y_train0 == y_train0[c]])
                    elif a == 1:
                        min_dist_same_dif_class[ds,a,b,c] = min_finder(big_distances[b,c][y_train1 == y_train1[c]])
                    else:
                        min_dist_same_dif_class[ds,a,b,c] = min_finder(big_distances[b,c][y_train2 == y_train2[c]])
                else:
                    if a == 0:
                        min_dist_same_dif_class[ds,a,b,c] = min_finder(big_distances[b,c][y_train0 != y_train0[c]])
                    elif a == 1:
                        min_dist_same_dif_class[ds,a,b,c] = min_finder(big_distances[b,c][y_train1 != y_train1[c]])
                    else:
                        min_dist_same_dif_class[ds,a,b,c] = min_finder(big_distances[b,c][y_train2 != y_train2[c]])            

### I have looped through the min_dist_same_dif_class matrix to calculate the alpha for every single point of all the different extended training sets. 
### In this loop I have accounted for the standard definitions of a/0 = $inf$ and 0/0 = 0

In [19]:
alpha = np.empty(3*len(big_train)*(len(big_train[0]))).reshape(3,len(big_train),(len(big_train[0])))
for i in range (3):
    for j in range (len(big_train)):
        for k in range (len(big_train[0])):
            if min_dist_same_dif_class[0,i,j,k] == 0:
                if min_dist_same_dif_class[1,i,j,k] == 0:
                    alpha[i,j,k] = 0
                else:
                    alpha[i,j,k] = np.inf
            else:
                alpha[i,j,k] = min_dist_same_dif_class[1,i,j,k]/min_dist_same_dif_class[0,i,j,k]

### To find the rank I have looped through every extended training data sets alpha's comparing them to all other alpha's of the same extended training dataset and I have increased the rank if the alpha is greater than or equal to another alpha. 
### This works because when deciding upon the rank in conformal prediciton if the two points are equal then the rank is considered to be the higher of the two. i.e if all alpha's were equal the rank would be the highest possible rank, n+1.

In [20]:
rank = np.ones(3*len(big_train)*(len(big_train[0]))).reshape(3,len(big_train),(len(big_train[0]))) 

for i in range (3):
    for j in range (len(big_train)):
        for k in range (len(alpha[0,0])):
            for l in range (len(alpha[0,0])):
                if k != l:
                    if alpha[i,j,k] >= alpha[i,j,l]:
                        rank[i,j,k] += 1

### Finally I have calculated the p-values for each different labelled test data point in every extended data set. 
### I've then used these p-values to calculate the average false p-value making sure to discount the true labels p-value instead of just the highest scoring p-value.

In [21]:
p_val = np.empty(len(big_train)*3).reshape(len(big_train),3)

for i in range (len(big_train)):
    for j in range (3):
        p_val[i,j] = rank[j,i,-1]/len(big_train[0])

In [22]:
false_p_val = 0
for i in range (len(big_train)):
    false_p_val += np.sum(p_val[i]) - p_val[i,y_test[i]]
average_false_p_val = false_p_val/(len(big_train)*2)

In [23]:
average_false_p_val

0.011411271541686084

Iris dataset average false p-value for the Nearest Neighbour conformal predictor = 0.0114 (3.s.f)

# Ionosphere data conformal prediction

### I carried out conformal prediction for the Ionosphere dataset in a very similar manner, the main alternative is that I used a second conformity measure 1/(distance to nearest sample of same class) as well as the one used previously to compare results. I'll state any other differences in the algorithm as they appear.
### The first slight difference being the size of the dimensions of the dataset will clearly be slighly different as seen below when creating the array of extended datasets.

In [24]:
big_train_io = np.empty((len(Xio_train)+1)*34*len(Xio_test)).reshape(len(Xio_test),(len(Xio_train)+1),34)
for i in range (len(Xio_test)):
    big_train_io[i] = np.r_[Xio_train,Xio_test[i].reshape(1,34)]

In [25]:
big_distances_io = np.empty(len(big_train_io)*len(big_train_io[0])*len(big_train_io[0])).reshape(len(big_train_io),len(big_train_io[0]),(len(big_train_io[0])))
for i in range (len(big_train_io)):
    for j in range (len(big_train_io[0])):
        for k in range (len(big_train_io[0])):
            if j != k:
                big_distances_io[i,j,k] = euc_dis(big_train_io[i,j],big_train_io[i,k])
            else:
                big_distances_io[i,j,k] = math.inf

### Due to there only being two labels in the Ionospere target dataset I only need to loop over two different potential labels for the test data points in each extended training dataset, instead of 3.

In [26]:
yio_train_neg = np.append(yio_train,-1)
yio_train_pos = np.append(yio_train,1)
min_dist_same_dif_class_io = np.empty(2*2*len(big_train_io)*(len(big_train_io[0]))).reshape(2,2,len(big_train_io),(len(big_train_io[0])))

for ds in range (2):
    for a in range (2):
        for b in range (len(big_train_io)):
            for c in range (len(big_train_io[0])):
                if ds == 0:
                    if a == 0:
                        min_dist_same_dif_class_io[ds,a,b,c] = min_finder(big_distances_io[b,c][yio_train_neg == yio_train_neg[c]])
                    else:
                        min_dist_same_dif_class_io[ds,a,b,c] = min_finder(big_distances_io[b,c][yio_train_pos == yio_train_pos[c]])
                else:
                    if a == 0:
                        min_dist_same_dif_class_io[ds,a,b,c] = min_finder(big_distances_io[b,c][yio_train_neg != yio_train_neg[c]])
                    else:
                        min_dist_same_dif_class_io[ds,a,b,c] = min_finder(big_distances_io[b,c][yio_train_pos != yio_train_pos[c]])

### The next few blocks of code show the two diffferent calculations of alpha. The first block uses the measure used previously which is:
(distance to the nearest sample of different class)/(distance to the nearest sample of same class)
### The second block of code uses the alternative conformity measure of:
1/(distance to the nearest value of same class)


In [27]:
alpha_io = np.empty(2*len(big_train_io)*(len(big_train_io[0]))).reshape(2,len(big_train_io),(len(big_train_io[0])))
for i in range (2):
    for j in range (len(big_train_io)):
        for k in range (len(big_train_io[0])):
            if min_dist_same_dif_class_io[0,i,j,k] == 0:
                if min_dist_same_dif_class_io[1,i,j,k] == 0:
                    alpha_io[i,j,k] = 0
                else:
                    alpha_io[i,j,k] = np.inf
            else:
                alpha_io[i,j,k] = min_dist_same_dif_class_io[1,i,j,k]/min_dist_same_dif_class_io[0,i,j,k]

In [28]:
alt_alpha_io = np.empty(2*len(big_train_io)*(len(big_train_io[0]))).reshape(2,len(big_train_io),(len(big_train_io[0])))
for i in range (2):
    for j in range (len(big_train_io)):
        for k in range (len(big_train_io[0])):
            if min_dist_same_dif_class_io[0,i,j,k] == 0:
                alt_alpha_io[i,j,k] = np.inf
            else:
                alt_alpha_io[i,j,k] = 1/min_dist_same_dif_class_io[0,i,j,k]

### The ranks of the alpha's of the original conformity measure and the alternative are both computed. These are used to calculate the different p-value's and subsequently the two average false p-value's

In [29]:
rank_io = np.ones(2*len(big_train_io)*(len(big_train_io[0]))).reshape(2,len(big_train_io),(len(big_train_io[0]))) 

for i in range (2):
    for j in range (len(big_train_io)):
        for k in range (len(alpha_io[0,0])):
            for l in range (len(alpha_io[0,0])):
                if k != l:
                    if alpha_io[i,j,k] >= alpha_io[i,j,l]:
                        rank_io[i,j,k] += 1

In [30]:
alt_rank_io = np.ones(2*len(big_train_io)*(len(big_train_io[0]))).reshape(2,len(big_train_io),(len(big_train_io[0]))) 

for i in range (2):
    for j in range (len(big_train_io)):
        for k in range (len(alt_alpha_io[0,0])):
            for l in range (len(alt_alpha_io[0,0])):
                if k != l:
                    if alt_alpha_io[i,j,k] >= alt_alpha_io[i,j,l]:
                        alt_rank_io[i,j,k] += 1

In [31]:
p_val_io = np.empty(len(big_train_io)*2).reshape(len(big_train_io),2)

for i in range (len(big_train_io)):
    for j in range (2):
        p_val_io[i,j] = rank_io[j,i,-1]/len(big_train_io[0])

In [32]:
alt_p_val_io = np.empty(len(big_train_io)*2).reshape(len(big_train_io),2)

for i in range (len(big_train_io)):
    for j in range (2):
        alt_p_val_io[i,j] = alt_rank_io[j,i,-1]/len(big_train_io[0])

### To be able to find the correct p-values index I had to change all the -1 labels to 0 as these 0 labels can be used as an index to subtract the correct p-value's of each test data point from the sum of the p-values.

In [33]:
for i in range (len(yio_test)):
    if yio_test[i] == -1:
        yio_test[i] = 0
false_p_val_io = 0
for i in range (len(big_train_io)):
    false_p_val_io += np.sum(p_val_io[i]) - p_val_io[i,int(yio_test[i])]
average_false_p_val_io = false_p_val_io/(len(big_train_io))

In [34]:
average_false_p_val_io

0.05720557851239672

In [35]:
alt_false_p_val_io = 0
for i in range (len(big_train_io)):
    alt_false_p_val_io += np.sum(alt_p_val_io[i]) - alt_p_val_io[i,int(yio_test[i])]
alt_average_false_p_val_io = alt_false_p_val_io/(len(big_train_io))

In [36]:
alt_average_false_p_val_io

0.2870179063360882

Ionosphere dataset average false p-value for the Nearest Neighbour conformal predictor = 0.0572 (3.s.f)

Using the alternate conformity measure the average false p-value = 0.287 (3.s.f)

# Results
(all to 3 significanct figures)

* the test error rate for Nearest Neighbour applied to iris.txt = 0.0526
* the test error rate for Nearest Neighbour applied to ionosphere.txt = 0.114
* the average false p-value for the Nearest Neighbour conformal predictor applied to iris.txt = 0.0114
* the average false p-value for the Nearest Neighbour conformal predictor applied to ionosphere.txt = 0.0572
* the average false p-value for the Nearest Neighbour alternative conformal predictor applied to ionosphere.txt = 0.287